# The simplest CMR search query
    GET https://cmr.earthdata.nasa.gov/search/collections

What can we do with this query?

In [1]:
import requests
import xml.dom.minidom

# The simplest search request possible
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections")

The default response is an xml document of collection result references. Let's pretty print the output.

In [3]:
# The default response is an xml document of result references. Let's pretty print the output
response_as_dom = xml.dom.minidom.parseString(response.text)
xml_reference_response = response_as_dom.toprettyxml()
print(xml_reference_response)

<?xml version="1.0" ?>
<results>
	<hits>28353</hits>
	<took>8</took>
	<references>
		<reference>
			<name>&quot;The Omnivores Dilemma&quot;: The Effect of Autumn Diet on Winter Physiology and Condition of Juvenile Antarctic Krill</name>
			<id>C1934541400-SCIOPS</id>
			<location>https://cmr.earthdata.nasa.gov:443/search/concepts/C1934541400-SCIOPS/1</location>
			<revision-id>1</revision-id>
		</reference>
		<reference>
			<name>'Latent reserves' within the Swiss NFI</name>
			<id>C1931110427-SCIOPS</id>
			<location>https://cmr.earthdata.nasa.gov:443/search/concepts/C1931110427-SCIOPS/5</location>
			<revision-id>5</revision-id>
		</reference>
		<reference>
			<name>(U-Th)/He ages from the Kukri Hills of southern Victoria Land</name>
			<id>C1214587974-SCIOPS</id>
			<location>https://cmr.earthdata.nasa.gov:443/search/concepts/C1214587974-SCIOPS/4</location>
			<revision-id>4</revision-id>
		</reference>
		<reference>
			<name>0.5 hour 1 M HCl extraction data for the Windmill Islands

How many collection results are there?

In [4]:
import xml.etree.ElementTree as et
doc = et.fromstring(response.text)
# Hits
print("Hits: " + doc.findtext('hits'))   


Hits: 28353


How many collection results were returned?

In [5]:
# No. of results returned
references = doc.findall('references/reference')
print("No. of results returned: " + str(len(references)))  


No. of results returned: 10


Where is the first collection result?

In [6]:
# First reference
print("First result reference: " + str(references[0].find('location').text))

First result reference: https://cmr.earthdata.nasa.gov:443/search/concepts/C1934541400-SCIOPS/1


What is the human-readable name of the first collection result?

In [7]:
# First name
print("First result name: " + str(references[0].find('name').text))

First result name: "The Omnivores Dilemma": The Effect of Autumn Diet on Winter Physiology and Condition of Juvenile Antarctic Krill


What is the unique ID (concept-id) of the first collection result?

In [8]:
print("First result concept id: " + str(references[0].find('id').text))

First result concept id: C1934541400-SCIOPS
